In [ ]:
import tensorflow as tf

In [ ]:
#设置多种优化器，根据参数选择
with tf.variable_scope("optimizer"):
    optimizer = self.config["optimizer"]
    if optimizer == "sgd":
        self.opt = tf.train.GradientDescentOptimizer(self.lr)
    elif optimizer == "adam":
        self.opt = tf.train.AdamOptimizer(self.lr)
    elif optimizer == "adgrad":
        self.opt = tf.train.AdagradOptimizer(self.lr)
    else:
        raise KeyError

In [ ]:
#方式一：opt.compute_gradients(self.loss)
grads_vars = self.opt.compute_gradients(self.loss) #根据损失，计算梯度，默认也是针对trainable_variables
#返回的是 A list of (gradient, variable) pairs.故而grads_vars是一个列表
capped_grads_vars = [[tf.clip_by_value(g, -self.config["clip"], self.config["clip"]), v]
                     for g, v in grads_vars] #截断梯度，前后都截断，防止梯度爆炸和消失，纯数值截断，比如大于5的梯度都设置为5，比-5小的设置为-5
#这里选择的是前后都截断，也可以选择下面的方式，即按一定比例变小
self.train_op = self.opt.apply_gradients(capped_grads_vars, self.global_step) #更新参数
#注意，这里需要把train_op放sess.run里面，才会去更新

In [ ]:
#方式二：tf.gradients(loss, params) 
params = tf.trainable_variables()
gradients = tf.gradients(loss, params) 
self.updates = []
#通过损失对需要更新的参数进行求梯度，更新选择的参数。
#tf.gradients最重要的即两个参数，一个即ys，这里即loss，另一个即xs，这里即要更新的params，即用ys对xs进行求偏导
clipped_gradients, norm = tf.clip_by_global_norm( gradients, clip_norm) #t_list[i] * clip_norm / max(global_norm, clip_norm)
                                                                        #global_norm = sqrt(sum([l2norm(t)**2 for t in t_list]))
self.gradient_norms.append(norm)
self.updates.append(opt.apply_gradients(zip(clipped_gradients, params))) #反向传播，更新参数
#同样的，这里的updates也要放在sess.run里面，即只有updates被运行了，参数才会被更新
#当然，最后一句话换成以下的也可以，同样是为了将梯度更新到参数，这里因为是多组，所以用一个列表接收
#self.train_op = opt.apply_gradients(zip(clipped_gradients, params))

In [ ]:
#另一种梯度截断tf.clip_by_norm
optimizer = tf.train.AdamOptimizer(learning_rate, beta1=0.5)
grads = optimizer.compute_gradients(loss)
for i, (g, v) in enumerate(grads):
    if g is not None:
        grads[i] = (tf.clip_by_norm(g, 5), v)  # 即对每个g做以下处理：`g * clip_norm / l2norm(g)`,相比global_norm少了一个max
train_op = optimizer.apply_gradients(grads)